In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf


In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
import pathlib

data_root = pathlib.Path.home().joinpath('Documents/Unimelb/Dissertation/test_Data/train_img/')
for item in data_root.iterdir():
  print(item)

/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/.DS_Store
/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/down_up
/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/up
/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/down


## 1. Load dataset

### 1) Filenames and labels

In [4]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)
all_image_paths = all_image_paths[:1000]
image_count = len(all_image_paths)
image_count

1000

In [5]:
all_image_paths[:5]

['/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/up/000697_20000619.png',
 '/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/up/000155_20070228.png',
 '/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/down_up/000783_20101115.png',
 '/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/up/002014_20041202.png',
 '/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/up/000655_20100525.png']

In [6]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['down', 'down_up', 'up']

In [7]:
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index

{'down': 0, 'down_up': 1, 'up': 2}

In [8]:
all_image_labels =[]
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:5])

First 10 labels indices:  [2, 2, 1, 2, 2]


Until now we have labels with type ndarray and image file paths of type list
Below we create a numpy array of filename and labels and save them as a .npy file.

In [9]:
import numpy as np
np.save('all_image_paths.npy', all_image_paths)
all_image_paths = np.asarray(all_image_paths)

# One hot vector representation of labels
all_image_labels = np.asarray(all_image_labels)
from keras.utils import to_categorical
all_image_labels = to_categorical(all_image_labels, num_classes=3)

# saving the y_labels_one_hot array as a .npy file
np.save('all_image_labels.npy', all_image_labels)

Using TensorFlow backend.


In [10]:
all_image_labels.shape

(1000, 3)

### 2) Split train and test data

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(all_image_paths, all_image_labels, test_size=0.2, random_state=6)

In [12]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(800,)
(200,)
(800, 3)
(200, 3)


In [13]:
y_train[1]

array([0., 1., 0.], dtype=float32)

## 2. Create Generator to save memory

In [14]:
from skimage.io import imread
from skimage.transform import resize
import keras
class Custom_Generator(keras.utils.Sequence) :
    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        
    def __len__(self):
        return (np.ceil(len(self.image_filenames)/
                        float(self.batch_size))).astype(np.int)
    
    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx*self.batch_size:
                                       (idx+1)*self.batch_size]
        batch_y = self.labels[idx*self.batch_size:(idx+1)*self.batch_size]
        
        x, y = np.array([resize(imread(file_name), (224, 224, 3))
               for file_name in batch_x])/255.0, np.array(batch_y)

        print(x[1])
        return x, y

In [15]:

batch_size = 32

training_batch_generator = Custom_Generator(x_train, y_train, batch_size)
validation_batch_generator = Custom_Generator(x_val, y_val, batch_size)

## 3. Build and train the model

### 1) Transfer learning 

In [16]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3

# base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
# base_model = VGG19(include_top=False, input_shape=(224, 224, 3))
# base_model = MobileNet(include_top=False, input_shape=(224, 224, 3))
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
# base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [17]:
FC_NUMS = 1024
IMAGE_SIZE = 224
FREEZE_LAYERS = 160
NUM_CLASSES = 3

In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(FC_NUMS, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

In [19]:
from tensorflow.keras.models import Model

# 构造完新的FC层，加入custom层
model = Model(inputs=base_model.input, outputs=prediction)
# 可观察模型结构
model.summary()
# 获取模型的层数
print("layer nums:", len(model.layers))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [20]:


# 除了FC层，靠近FC层的一部分卷积层可参与参数训练，
# 一般来说，模型结构已经标明一个卷积块包含的层数，
# 在这里我们选择FREEZE_LAYERS为17，表示最后一个卷积块和FC层要参与参数训练

for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True
for layer in model.layers:
    print("layer.trainable:", layer.trainable)

layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.trainable: False
layer.train

In [21]:
from tensorflow.keras.optimizers import SGD, Adam

learing_rate = 1e-3
model.compile(optimizer=SGD(lr=learing_rate),
              loss='categorical_crossentropy', metrics=['accuracy'])

### 3) Prepare the callback module

In [22]:
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))

In [23]:
callbacks = [
# 当验证集上的损失“val_loss”连续两个训练回合（epoch）都没有变化，则提前结束训练
tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_loss'),
# 使用TensorBoard保存训练的记录，保存到“./logs”目录中
tf.keras.callbacks.TensorBoard(log_dir='./logs'),
tf.keras.callbacks.LearningRateScheduler(scheduler)]

### 4) Train with generator

In [24]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

train_dir = '/Users/cyril/Documents/Unimelb/Dissertation/test_Data/train_img/'
 
num_epochs = 2
batch_size = 32
 
data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=0.1)
 
# classes: 可选参数,为子文件夹的列表,如['dogs','cats']默认为None.
# 若未提供,则该类别列表将从directory下的子文件夹名称/结构自动推断。
# 每一个子文件夹都会被认为是一个新的类。(类别的顺序将按照字母表顺序映射到标签值)。
# 通过属性class_indices可获得文件夹名与类的序号的对应字典。
# 本例中使用默认的参数，表示按数字或字母升序，对应类的序号
train_generator = data_gen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=batch_size,
                                               class_mode='categorical', subset='training')
validation_generator = data_gen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=batch_size,
                                               class_mode='categorical', subset='validation')

 
# 训练模型
model.fit_generator(generator=train_generator,
                    epochs=num_epochs,
                    verbose=2,
                    validation_data=validation_generator,
                    callbacks=callbacks)

Found 13500 images belonging to 3 classes.
Found 1500 images belonging to 3 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 422 steps, validate for 47 steps
Epoch 1/2
422/422 - 704s - loss: 1.1264 - accuracy: 0.3435 - val_loss: 1.3853 - val_accuracy: 0.3327
Epoch 2/2
422/422 - 684s - loss: 1.0964 - accuracy: 0.3813 - val_loss: 1.3854 - val_accuracy: 0.3280


In [25]:
export_path = tf.contrib.saved_model.save_keras_model(model, 'keras_export')
print("Model exported to: ", export_path)

AttributeError: module 'tensorflow' has no attribute 'contrib'